In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
df=pd.read_csv('../input/covid-vaccination-india-district-wise-data/vaccine_doses_statewise.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum().sum()

No value is empty...So we can start...

**Firstly I will rename the last column as it is the data of total vaccinated till that date from 16 Jan...**

In [ ]:
def rename():
    df.rename(columns={df.columns[-1]: 'Total Vaccinated'}, inplace=True)

rename()

In [ ]:
df.loc[df['State']=='Miscellaneous']

I am removing the 36th row as we don't know which part of India is present in this and we don't know that which part's population we have to use and it will not affect our work as it is used in the sum with other states which gave the 37th row(i.e Total)...

In [ ]:
df=df.drop(36,axis=0)

In [ ]:
#We have removed the row so we have to reset the index...

df=df.reset_index(drop=True)

In [ ]:
df

Now, I will take and use the population data of each state and will put it in the dataframe and use it to find the percentage of people vaccinated in each state and total people vaccinated in India...

In [ ]:
#So this is our population data which we have taken from internet i.e. population of each state and the last value is the total population of India..

population=[420000,54400000,1592000,36000000,127000000,1169000,29800000,620000,19000000,1600000,64300000,28500000,7500000,13800000,39200000,68300000,36000000,290000,74151,87000000,124300000,3200000,3400000,1255000,2300000,46900000,1432000,30000000,82500000,700000,78500000,39800000,4200000,240000000,11400000,100000000,1386452151]

In [ ]:
#Now we will add it in the dataframe as a column...

df['Total Population']=population

In [ ]:
df

In [ ]:
#Now I will add another column after finding %age vaccinated using last two columns i.e Total Vaccinated and Total Population...

df['%age Vaccinated']=(df['Total Vaccinated'] / df['Total Population']) * 100

In [ ]:
df.head()

In [ ]:
#Now I will select only two columns i.e State and %age Vaccinated so that I can use it in plotting the graph...

Vacci_Per=df[['State','%age Vaccinated']].drop(36)

In [ ]:
Vacci_Per

In [ ]:
#Now I will make another variable so that I can use the value of %age vaccinated in India...
Total=df[['State','%age Vaccinated']]
Total=Total.loc[Total.State=='Total']
Total

In [ ]:
#Plotting the Graph

plt.figure(figsize=(15,8))
plt.bar(Vacci_Per['State'],Vacci_Per['%age Vaccinated'],color=(Vacci_Per.State=='Punjab').map({True:'#8C000F',False:'#929591'}))
plt.axhline(Total['%age Vaccinated'].values, color = '#008000', ls = '--', lw = 5,label ='Total %age vaccinated in India')
plt.xlabel('States')
plt.ylabel('Percentage')
plt.title('%age of people vaccinated in each state')
plt.xticks(rotation=90)
plt.legend(edgecolor = 'black')
plt.show()

**Now, I will read the csv file agail as i wanted to use it in some other manner...**

In [ ]:
df=pd.read_csv("../input/covid-vaccination-india-district-wise-data/vaccine_doses_statewise.csv")

In [ ]:
df=df.set_index('State').T

In [ ]:
df=df.reset_index()

In [ ]:
df.head()

So I have obtained the dataframe in the manner as shown above. Now I will drop Miscellaneous and Total column as I don't need them...

In [ ]:
df=df.drop(['Miscellaneous','Total'],axis=1)

In [ ]:
df=df.rename(columns={'index':'Date'})

In [ ]:
df.head()

In [ ]:
date=df['Date']

Now I will find the vaccine doses which were given each day as our data was not sorted and each day the doses of the previous day was added to each next day...

In [ ]:
df=df.iloc[:, 1:]-df.shift().fillna(0).iloc[:, 1:]

And I will insert the date column back as I don't wanted to apply the above...

In [ ]:
df.insert(0,'Date',date)

So we got the dataframe...

In [ ]:
df.head()

In [ ]:
#I will remove the State as shown above the index...

df = df.rename_axis(None, axis=1)

In [ ]:
#Plotting Punjab's Graph

df[['Date','Punjab']].set_index(['Date']).plot(figsize=(15,8))
plt.show()

# Rolling Mean

In [ ]:
roll=df['Punjab'].rolling(10, min_periods=10).mean()
date=df['Date']

In [ ]:
date,roll[9:].plot(figsize=(15,8))
plt.show()

In [ ]:
df.head()

In [ ]:
days=pd.to_datetime(df['Date'],dayfirst=True)
days

In [ ]:
#Plotting rolling mean with some other features which can be shown in the graph...

df2 = pd.concat([days, roll], 1)[9:].set_index(['Date'])
df2.plot(figsize = (15, 8), linestyle = '--')
plt.vlines(['2021-05-01 00:00:00', '2021-06-21 00:00:00'], ymin = 0.03, ymax = 120000)
plt.axvspan('2021-05-01 00:00:00', '2021-06-21 00:00:00', color = 'maroon', alpha = 0.2, ymin = 0.04, ymax = 110000)
plt.legend(["Punjab's vaccination drive", "stringent policy of center", "May 1 until June 21, 2021"])
plt.show()

In [ ]:
df.head()

# Exponentially Weighted Moving Average (EWMA)

In [ ]:
ewm=df[['Punjab','Haryana']].ewm(200).mean()

**Punjab VS Haryana**

In [ ]:
date,ewm.set_index(df['Date']).plot(figsize=(15,8))
plt.show()

# Serial Correlation

In [ ]:
def SerialCorrelation(series, lag = 1):
    xs = series[lag:]
    ys = series.shift(lag)[lag:]
    corr = correlation(xs, ys)
    return corr

def correlation(x, y):
    return covariance(x, y) / (np.std(x) * np.std(y))

def covariance(x, y):
    xbar = np.mean(x)
    ybar = np.mean(y)
    n = len(x)
    
    xs = [x_i - xbar for x_i in x]
    ys = [y_i - ybar for y_i in y]
    
    return np.dot(xs, ys) / n

In [ ]:
lags = [1, 7, 30]
items = ['day', 'week', 'month']

for i, j in zip(lags, items):
    corr = SerialCorrelation(df['Punjab'], lag = i)
    print('{}: {}'.format(j, corr))

In [ ]:
lags = range(1,30)
corr = []

for i in lags:
    cor = SerialCorrelation(df['Punjab'], lag = i)
    corr.append(cor)
    
plt.plot(lags, corr, label = 'Serial Correlation(Punjab)')
plt.gcf().set_size_inches(12, 6)
plt.legend()
plt.show()